# RecDP LLM - PII Removal

# Get started

## 1. Install pyrecdp and dependencies

In [ ]:
! DEBIAN_FRONTEND=noninteractive apt-get install -y openjdk-8-jre
! pip install pyrecdp --pre
# ! pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

## 2. Prepare your own data

In [ ]:
%mkdir -p /tmp/pii_remove
%cd /tmp/pii_remove
!wget https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/llm_data/tiny_c4_sample_for_pii.jsonl

## 3. PII Detect and Redact

In [9]:
from pyrecdp.primitives.llmutils import pii_remove
from pyspark.sql import SparkSession
from pyrecdp.core.cache_utils import RECDP_MODELS_CACHE
import os

spark = SparkSession.builder.getOrCreate()
input_dataset = spark.read.json("file:///tmp/pii_remove/tiny_c4_sample_for_pii.jsonl")

model_root_path = os.path.join(RECDP_MODELS_CACHE, "huggingface")
output_dataset = pii_remove(dataset=input_dataset,text_column="text", model_root_path=model_root_path, show_secret_column=True)
output_dataset.show()


/root/miniforge3/envs/recdp/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


+--------------------+--------------------+--------------------+------------------+--------------------+
|         __SECRETS__|                meta|                text|is_modified_by_pii|             secrets|
+--------------------+--------------------+--------------------+------------------+--------------------+
|[{start=23, end=4...|{"timestamp":"201...|This is a test em...|              true|[{'tag': 'EMAIL',...|
|[{start=28, end=4...|{"timestamp":"201...|This is a test ph...|              true|[{'tag': 'PHONE_N...|
|[{start=27, end=3...|{"timestamp":"201...|This is a test ip...|              true|[{'tag': 'IP_ADDR...|
|[{start=17, end=2...|                null|Hello, my name is...|              true|[{'tag': 'NAME', ...|
|                  []|{"timestamp":"201...|SCRAPPY is a supe...|             false|                  []|
|[{start=453, end=...|{"timestamp":"201...|POLL: Is The SEC ...|              true|[{'tag': 'NAME', ...|
|[{start=21, end=4...|{"timestamp":"201...|Detailed rac